In [119]:
# import os
# os.system('pip uninstall torch torchvision torchaudio torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv pyg-lib -y')

# !pip install torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121

# !pip install torch-scatter==2.1.2+pt23cu121 torch-sparse==0.6.18+pt23cu121 torch-cluster==1.6.3+pt23cu121 torch-spline-conv==1.2.2+pt23cu121 -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

# !pip install torch-geometric
# !pip install pyg-lib -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

# import os

In [120]:
print("Starting")
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import os
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Starting
Using device: cuda


In [121]:
def convert_hetero_to_homogeneous(hetero_data):
    """
    Converts a HeteroData object to a homogeneous graph format that a standard
    GCNConv layer can process. It also returns a mask to identify influencers.
    """
    # Create a single large feature matrix by concatenating features of all node types
    x = torch.cat([hetero_data[node_type].x for node_type in hetero_data.node_types], dim=0)
    
    # Create offsets to map original node indices to the new homogeneous indices
    offsets = {}
    current_offset = 0
    for node_type in hetero_data.node_types:
        offsets[node_type] = current_offset
        current_offset += hetero_data[node_type].num_nodes
        
    # Combine edge indices, adjusting them with the offsets
    edge_indices = []
    for src, _, dst in hetero_data.edge_types:
        edge_index = hetero_data[(src, _, dst)].edge_index
        edge_index[0] += offsets[src]
        edge_index[1] += offsets[dst]
        edge_indices.append(edge_index)
    edge_index = torch.cat(edge_indices, dim=1)

    # Create a boolean mask to identify which nodes in the homogeneous graph are influencers
    influencer_mask = torch.zeros(x.size(0), dtype=torch.bool)
    influencer_offset = offsets['influencer']
    num_influencers = hetero_data['influencer'].num_nodes
    influencer_mask[influencer_offset : influencer_offset + num_influencers] = True
    
    return x, edge_index, influencer_mask

class PaperGNNEncoder(nn.Module):
    """The simple GCN Encoder block from the paper."""
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

print("Helper functions and GNN Encoder defined.")

Helper functions and GNN Encoder defined.


In [122]:
class FastInfluencerRank(nn.Module):
    """Phase 2: Lightning-fast model that only processes pre-computed embeddings"""
    def __init__(self, gnn_out, rnn_hidden, dropout=0.5):
        super().__init__()
        # NO GNN - only RNN and final layers for speed
        self.rnn = nn.GRU(input_size=gnn_out, hidden_size=rnn_hidden, batch_first=True)
        self.attention = nn.Linear(rnn_hidden, 1)
        self.fc1 = nn.Linear(rnn_hidden, rnn_hidden // 2)
        self.fc2 = nn.Linear(rnn_hidden // 2, 1)
        self.dropout = nn.Dropout(dropout)
    
    def create_attention_mask(self, lengths, max_len):
        batch_size = len(lengths)
        mask = torch.arange(max_len, device=lengths.device).expand(batch_size, max_len) < lengths.unsqueeze(1)
        return mask
    
    def forward(self, precomputed_sequences, valid_lengths):
        """
        precomputed_sequences: [batch_size, max_months, embedding_dim]
        valid_lengths: [batch_size] - number of valid months per influencer
        """
        if precomputed_sequences.size(0) == 0:
            return torch.tensor([], device=precomputed_sequences.device)
        
        # RNN processing
        rnn_out, _ = self.rnn(precomputed_sequences)
        
        # Attention with masking
        attention_scores = self.attention(rnn_out).squeeze(-1)
        attention_mask = self.create_attention_mask(valid_lengths, precomputed_sequences.size(1))
        attention_scores = attention_scores.masked_fill(~attention_mask, float('-inf'))
        attention_weights = torch.softmax(attention_scores, dim=1).unsqueeze(-1)
        
        # Weighted sum
        context = torch.sum(attention_weights * rnn_out, dim=1)
        
        # Final prediction
        x = F.relu(self.fc1(context))
        x = self.dropout(x)
        scores = self.fc2(x)
        
        return scores.squeeze(-1)

print("Fast two-phase model architecture defined.")

Fast two-phase model architecture defined.


In [123]:
def precompute_all_gnn_embeddings():
    """Phase 1: Run GNN once on all graphs to 'bake' embeddings"""
    print("=== PHASE 1: Pre-computing GNN embeddings ===")
    
    # Hyperparameters
    NODE_FEATURES = FINAL_FEATURE_DIM
    GNN_HIDDEN = 128
    GNN_OUT = 128
    
    # Initialize just the GNN encoder
    gnn_encoder = PaperGNNEncoder(NODE_FEATURES, GNN_HIDDEN, GNN_OUT).to(device)
    gnn_encoder.eval()
    
    monthly_embeddings = {}
    
    with torch.no_grad():
        for month_idx, (x, edge_index, influencer_mask) in enumerate(tqdm(homogeneous_graphs, desc="Processing months")):
            # Move data to GPU
            x = x.to(device)
            edge_index = edge_index.to(device)
            
            # Get embeddings for all nodes
            all_embeddings = gnn_encoder(x, edge_index)
            
            # Extract only influencer embeddings and move to CPU
            influencer_embeddings = all_embeddings[influencer_mask].cpu()
            monthly_embeddings[month_idx] = influencer_embeddings
            
            print(f"Month {month_idx}: {influencer_embeddings.shape[0]} influencer embeddings")
    
    # Save embeddings
    torch.save(monthly_embeddings, 'precomputed_embeddings.pt')
    print("Phase 1 complete! Embeddings saved to 'precomputed_embeddings.pt'")
    
    # Clear GPU memory
    del gnn_encoder
    torch.cuda.empty_cache()
    
    return monthly_embeddings

# Run Phase 1
monthly_embeddings = precompute_all_gnn_embeddings()

=== PHASE 1: Pre-computing GNN embeddings ===


Processing months:  25%|██▌       | 3/12 [00:00<00:00, 29.30it/s]

Month 0: 3258 influencer embeddings
Month 1: 4465 influencer embeddings
Month 2: 5973 influencer embeddings
Month 3: 2745 influencer embeddings
Month 4: 2556 influencer embeddings
Month 5: 4136 influencer embeddings
Month 6: 3831 influencer embeddings
Month 7: 2998 influencer embeddings


Processing months:  75%|███████▌  | 9/12 [00:00<00:00, 43.19it/s]

Month 8: 3549 influencer embeddings


Processing months: 100%|██████████| 12/12 [00:00<00:00, 36.93it/s]

Month 9: 5545 influencer embeddings
Month 10: 5180 influencer embeddings
Month 11: 4808 influencer embeddings
Phase 1 complete! Embeddings saved to 'precomputed_embeddings.pt'


In [124]:
GRAPH_DIR = '/kaggle/input/influencer-rank/graphs'
all_graphs_data = [torch.load(os.path.join(GRAPH_DIR, f), map_location='cpu', weights_only=False) 
                   for f in sorted(os.listdir(GRAPH_DIR)) if f.endswith('.pt')]
print(f"Loaded {len(all_graphs_data)} monthly data packages.")

# --- Feature preparation ---
for data_package in all_graphs_data:
    graph = data_package['graph']
    if 'x' in graph['influencer']:
        graph['influencer'].x_original = graph['influencer'].x.clone()
        # columns_to_keep = [0, 1, 2, 3, 4]
        columns_to_keep = list(range(min(5, graph['influencer'].x_original.shape[1])))
        graph['influencer'].x = graph['influencer'].x[:, columns_to_keep]

# --- Feature Normalization ---
print("Normalizing features...")
all_features_list = []
for data_package in all_graphs_data:
    if 'x' in data_package['graph']['influencer'] and data_package['graph']['influencer'].x.numel() > 0:
        all_features_list.append(data_package['graph']['influencer'].x.numpy())

if all_features_list:
    all_features_combined = np.vstack(all_features_list)
    scaler = StandardScaler()
    scaler.fit(all_features_combined)

    for data_package in all_graphs_data:
        if 'x' in data_package['graph']['influencer'] and data_package['graph']['influencer'].x.numel() > 0:
            normalized_features = scaler.transform(data_package['graph']['influencer'].x.numpy())
            data_package['graph']['influencer'].x = torch.FloatTensor(normalized_features)
print("Feature normalization complete.")

# --- Zero-pad other node types ---
FINAL_FEATURE_DIM = all_graphs_data[0]['graph']['influencer'].x.shape[1]
for data_package in all_graphs_data:
    for node_type in ['hashtag', 'user', 'object']:
        num_nodes = data_package['graph'][node_type].num_nodes
        data_package['graph'][node_type].x = torch.zeros(num_nodes, FINAL_FEATURE_DIM)
print("All data loaded and prepared.")

# --- Pre-convert graphs to homogeneous format (for speed) ---
print("Pre-converting graphs to homogeneous format...")
homogeneous_graphs = [convert_hetero_to_homogeneous(data['graph']) for data in tqdm(all_graphs_data)]

Loaded 12 monthly data packages.
Normalizing features...
Feature normalization complete.
All data loaded and prepared.
Pre-converting graphs to homogeneous format...


100%|██████████| 12/12 [00:00<00:00, 482.75it/s]


In [125]:
def prepare_precomputed_batch(monthly_embeddings, batch_names, local_maps):
    """Prepare batch using pre-computed embeddings"""
    batch_sequences = []
    valid_lengths = []
    
    for name in batch_names:
        sequence = []
        for month_idx in range(len(local_maps)):
            if name in local_maps[month_idx]:
                local_idx = local_maps[month_idx][name]
                if local_idx < monthly_embeddings[month_idx].shape[0]:
                    embedding = monthly_embeddings[month_idx][local_idx]
                    sequence.append(embedding)
        
        if sequence:
            valid_lengths.append(len(sequence))
            batch_sequences.append(torch.stack(sequence))
    
    if not batch_sequences:
        return torch.tensor([]), torch.tensor([])
    
    # Pad sequences
    padded_sequences = pad_sequence(batch_sequences, batch_first=True, padding_value=0.0)
    valid_lengths = torch.tensor(valid_lengths)
    
    return padded_sequences, valid_lengths

print("Fast batch preparation function defined.")

Fast batch preparation function defined.


In [126]:
# Hyperparameters
GNN_OUT = 128  
RNN_HIDDEN = 128
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# Initialize fast model
model = FastInfluencerRank(GNN_OUT, RNN_HIDDEN).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

def listwise_ranking_loss(y_pred, y_true):
    y_pred_diff = y_pred.unsqueeze(1) - y_pred.unsqueeze(0)
    y_true_diff = y_true.unsqueeze(1) - y_true.unsqueeze(0)
    mask = (y_true_diff > 0).float()
    loss = torch.log(1 + torch.exp(-y_pred_diff)) * mask
    return torch.sum(loss) / torch.sum(mask).clamp(min=1)

# Data splits
train_local_maps = [d['maps']['influencer'] for d in all_graphs_data[:10]]
val_local_maps = [d['maps']['influencer'] for d in all_graphs_data[:11]]

train_influencers = sorted(list(set(name for month_map in train_local_maps for name in month_map.keys())))
val_influencers = sorted(list(set(name for month_map in val_local_maps for name in month_map.keys())))

print(f"=== PHASE 2: Lightning-fast training ===")
print(f"Training influencers: {len(train_influencers)}")

# Sanity check
print("\n--- Quick Sanity Check ---")
test_batch_names = train_influencers[:3]
test_sequences, test_lengths = prepare_precomputed_batch(monthly_embeddings, test_batch_names, train_local_maps)

y_true_target_graph = all_graphs_data[9]['graph']['influencer']
y_true_target_map = all_graphs_data[9]['maps']['influencer']
valid_names = [name for name in test_batch_names if name in y_true_target_map]
y_true_indices = [y_true_target_map[name] for name in valid_names]
y_true_batch = y_true_target_graph.x_original[y_true_indices, 5].to(device)

model.train()
scores = model(test_sequences.to(device), test_lengths.to(device))
print(f"Scores shape: {scores.shape}, Y_true shape: {y_true_batch.shape}")

if scores.shape[0] == y_true_batch.shape[0]:
    print("✅ Sanity check passed!")
else:
    print("❌ Shape mismatch!")

=== PHASE 2: Lightning-fast training ===
Training influencers: 6113

--- Quick Sanity Check ---
Scores shape: torch.Size([3]), Y_true shape: torch.Size([3])
✅ Sanity check passed!


In [127]:
def ndcg_score(y_true, y_pred, k=50):
    if len(y_true) < 2: return 0.0
    order = torch.argsort(y_pred, descending=True)
    y_true_sorted = y_true[order]
    dcg = sum(y_true_sorted[i] / math.log2(i + 2) for i in range(min(k, len(y_true_sorted))))
    ideal_order = torch.argsort(y_true, descending=True)
    y_true_ideal_sorted = y_true[ideal_order]
    idcg = sum(y_true_ideal_sorted[i] / math.log2(i + 2) for i in range(min(k, len(y_true_ideal_sorted))))
    return dcg / idcg if idcg > 0 else 0.0

print("Starting lightning-fast training...")
best_val_ndcg = -1
NUM_EPOCHS = 250

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_losses = []
    
    # Shuffle influencers
    shuffled_influencers = torch.randperm(len(train_influencers))
    
    for i in tqdm(range(0, len(train_influencers), BATCH_SIZE), desc=f"Epoch {epoch+1}", leave=False):
        batch_indices = shuffled_influencers[i:i+BATCH_SIZE]
        batch_names = [train_influencers[j] for j in batch_indices]
        
        # Get ground truth
        y_true_target_graph = all_graphs_data[9]['graph']['influencer']
        y_true_target_map = all_graphs_data[9]['maps']['influencer']
        valid_names = [name for name in batch_names if name in y_true_target_map]
        
        if len(valid_names) < 2:
            continue
            
        y_true_indices = [y_true_target_map[name] for name in valid_names]
        y_true_batch = y_true_target_graph.x_original[y_true_indices, 5].to(device)
        
        # Prepare batch (lightning fast!)
        sequences, lengths = prepare_precomputed_batch(monthly_embeddings, valid_names, train_local_maps)
        
        if sequences.size(0) == 0:
            continue
            
        optimizer.zero_grad()
        scores = model(sequences.to(device), lengths.to(device))
        
        if scores.shape[0] != y_true_batch.shape[0]:
            continue
            
        loss = listwise_ranking_loss(scores, y_true_batch)
        
        if not torch.isnan(loss):
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_losses.append(loss.item())
    
    # Validation
    if epoch_losses:
        model.eval()
        with torch.no_grad():
            # Sample validation influencers for speed
            val_sample = val_influencers[:1000]  # Sample for speed
            y_true_val_graph = all_graphs_data[10]['graph']['influencer']
            y_true_val_map = all_graphs_data[10]['maps']['influencer']
            valid_val_names = [name for name in val_sample if name in y_true_val_map]
            
            if valid_val_names:
                y_true_val_indices = [y_true_val_map[name] for name in valid_val_names]
                y_true_val = y_true_val_graph.x_original[y_true_val_indices, 5]
                
                val_sequences, val_lengths = prepare_precomputed_batch(monthly_embeddings, valid_val_names, val_local_maps)
                
                if val_sequences.size(0) > 0:
                    val_scores = model(val_sequences.to(device), val_lengths.to(device))
                    if val_scores.shape[0] == y_true_val.shape[0]:
                        val_ndcg = ndcg_score(y_true_val, val_scores.cpu())
                    else:
                        val_ndcg = 0.0
                else:
                    val_ndcg = 0.0
            else:
                val_ndcg = 0.0
        
        avg_loss = np.mean(epoch_losses)
        print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}, Val NDCG@50: {val_ndcg:.4f}")
        
        if val_ndcg > best_val_ndcg:
            best_val_ndcg = val_ndcg
            torch.save(model.state_dict(), 'best_fast_model.pth')

print(f"Training complete! Best validation NDCG@50: {best_val_ndcg:.4f}")

Starting lightning-fast training...


Epoch 1 - Loss: 0.6844, Val NDCG@50: 0.3201


Epoch 2 - Loss: 0.6802, Val NDCG@50: 0.2877


Epoch 3 - Loss: 0.6797, Val NDCG@50: 0.3083


Epoch 4 - Loss: 0.6770, Val NDCG@50: 0.3267


...

Epoch 246 - Loss: 0.3297, Val NDCG@50: 0.5646


Epoch 247 - Loss: 0.3275, Val NDCG@50: 0.6191


Epoch 248 - Loss: 0.3304, Val NDCG@50: 0.6353


Epoch 249 - Loss: 0.3245, Val NDCG@50: 0.5986


Epoch 250 - Loss: 0.3229, Val NDCG@50: 0.5736
Training complete! Best validation NDCG@50: 0.6603
